In [1]:
import requests
import pandas as pd
import geopandas as gpd

In [2]:
api_key= 'Q0dGwScJ.EdgL1P5wLue7rrlFaIjQsr27LwbZyjzu'
url= 'https://api.patentsview.org/patents/query'
params= {
    'q': '{"_gte": {"patent_date":"2023-03-31"}}', #end date is march 31, 2024
    'f': '["patent_number", "patent_title", "assignee_city", "assignee_state", "assignee_country"]',
    'o': '{"per_page":"10000"}',
    'api_key': api_key
}

response = requests.get(url, params=params)
patent_data = response.json()


In [3]:
if 'patents' in patent_data:
    for patent in patent_data['patents']:
        assignees = patent.get('assignees', [])
        if assignees:
            assignee = assignees[0]
            latitude = assignee.get('assignee_latitude')
            longitude = assignee.get('assignee_longitude')
            print(f"Patent Number: {patent['patent_number']}, Title: {patent['patent_title']}, Latitude: {latitude}, Longitude: {longitude}")
else:
    print("No patents found in the response.")


Patent Number: 11617292, Title: Method for controlling the operation of an attachment, Latitude: None, Longitude: None
Patent Number: 11617293, Title: Hitch assembly for a work vehicle, Latitude: None, Longitude: None
Patent Number: 11617294, Title: Orientation control system for an agricultural implement, Latitude: None, Longitude: None
Patent Number: 11617295, Title: Dynamic supplemental downforce control system for planter row units, Latitude: None, Longitude: None
Patent Number: 11617296, Title: Agricultural machine equipped with a distribution tie-rod, Latitude: None, Longitude: None
Patent Number: 11617297, Title: Agricultural vehicle-trailer combination, Latitude: None, Longitude: None
Patent Number: 11617298, Title: Agricultural product delivery applicator with a pneumatic conveying system having a distributor assembly, Latitude: None, Longitude: None
Patent Number: 11617299, Title: System and method for regulating and dispensing seeds, fertilizer, and other items, Latitude: No

In [4]:
print(patent_data['patents'][0])

{'patent_number': '11617292', 'patent_title': 'Method for controlling the operation of an attachment', 'assignees': [{'assignee_city': 'Moline', 'assignee_state': 'IL', 'assignee_country': 'US', 'assignee_key_id': 110}]}


In [5]:
us_patents= []
i= 0
if 'patents' in patent_data:
    for patent in patent_data['patents']:
        assignees = patent.get('assignees', [])
        if assignees:
            assignee = assignees[0]
            city = assignee.get('assignee_city')
            state = assignee.get('assignee_state')
            country= assignee.get('assignee_country')
            i= i + 1
            #print(f"Patent Number {i}: {patent['patent_number']}, Title: {patent['patent_title']}, City: {city}, State: {state}, Country: {country}")

            if country== 'US':
                us_patents.append(patent)
                
else:
    print("No patents found in the response.")

print(len(us_patents))

4797


In [6]:
us_patents[0]

{'patent_number': '11617292',
 'patent_title': 'Method for controlling the operation of an attachment',
 'assignees': [{'assignee_city': 'Moline',
   'assignee_state': 'IL',
   'assignee_country': 'US',
   'assignee_key_id': 110}]}

In [7]:
df = pd.json_normalize(us_patents, 
                       record_path='assignees', 
                       meta=['patent_number', 'patent_title'], 
                       errors='ignore')

In [8]:
df

,assignee_city,assignee_state,assignee_country,assignee_key_id,patent_number,patent_title
0,Moline,IL,US,110,11617292,Method for controlling the operation of an att...
1,Moline,IL,US,110,11617293,Hitch assembly for a work vehicle
2,New Holland,PA,US,277,11617294,Orientation control system for an agricultural...
3,Tremont,IL,US,11109,11617295,Dynamic supplemental downforce control system ...
4,Moline,IL,US,110,11617297,Agricultural vehicle-trailer combination
...,...,...,...,...,...,...
4879,San Jose,CA,US,17671,11627372,Media channel identification with multi-match ...
4880,Washington,DC,US,326360,11627373,Systems and methods for providing survey data ...
4881,West Hollywood,CA,US,3430,11627375,Methods and systems for generating and providi...
4882,Suwanee,GA,US,5102,11627377,Set-top box ambiance and notification controller


In [9]:
city_counts= df['assignee_city'].value_counts()

In [10]:
city_count_list= city_counts.reset_index()
city_count_list.columns= ['city', 'count']

In [11]:
city_count_list= city_count_list.to_dict(orient= 'records')

In [12]:
city_count_list

[{'city': 'San Jose', 'count': 159},
 {'city': 'Armonk', 'count': 156},
 {'city': 'San Francisco', 'count': 154},
 {'city': 'Santa Clara', 'count': 153},
 {'city': 'San Diego', 'count': 140},
 {'city': 'Mountain View', 'count': 125},
 {'city': 'New York', 'count': 122},
 {'city': 'Seattle', 'count': 87},
 {'city': 'Chicago', 'count': 72},
 {'city': 'Houston', 'count': 71},
 {'city': 'Dearborn', 'count': 70},
 {'city': 'Cupertino', 'count': 67},
 {'city': 'Boise', 'count': 63},
 {'city': 'Redmond', 'count': 63},
 {'city': 'Charlotte', 'count': 59},
 {'city': 'Atlanta', 'count': 58},
 {'city': 'Cambridge', 'count': 53},
 {'city': 'Austin', 'count': 50},
 {'city': 'Menlo Park', 'count': 49},
 {'city': 'Palo Alto', 'count': 48},
 {'city': 'Wilmington', 'count': 47},
 {'city': 'Boston', 'count': 47},
 {'city': 'McLean', 'count': 44},
 {'city': 'Santa Monica', 'count': 36},
 {'city': 'Dallas', 'count': 36},
 {'city': 'Washington', 'count': 32},
 {'city': 'Sunnyvale', 'count': 30},
 {'city': 

In [14]:
!pip install matplotlib geopandas contextily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 36.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 6.4 MB/s eta 0:00:00


In [15]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [17]:
cc_df= pd.DataFrame(city_count_list)
cc_df

,city,count
0,San Jose,159
1,Armonk,156
2,San Francisco,154
3,Santa Clara,153
4,San Diego,140
...,...,...
864,Derry,1
865,Havertown,1
866,Ayer,1
867,Grafton,1


In [25]:
cc_df.to_csv("Econ_Research_City_List.csv", index= False)

In [19]:
import pandas as pd
from geopy.geocoders import Nominatim
import time


In [21]:
# Initialize geocoder
#geolocator = Nominatim(user_agent= "geoapiExercises")
geolocator = Nominatim(user_agent= "geoapiExercises")

# Function to get latitude and longitude
def get_lat_lon(city):
    try:
        location = geolocator.geocode(city + ", US")
        return location.latitude, location.longitude
    except AttributeError:
        return None, None

# Add latitude and longitude columns
cc_df[['latitude', 'longitude']] = cc_df['city'].apply(get_lat_lon).apply(pd.Series)

# Optional: Pause between requests to avoid hitting rate limits
time.sleep(1)

# Display the updated DataFrame
print(cc_df)

GeocoderInsufficientPrivileges: Non-successful status code 403